In [ ]:
from Database import ChromaDB
chroma = ChromaDB()
vector_store = chroma.get_vector_store(collection_name='bge_m3_collection', embedding_model='BAAI/bge-m3')
chroma.current_collection

In [ ]:
import json
with open("./競賽資料集/dataset/preliminary/questions_example.json", "r") as q:
    question_set = json.load(q)

# question_set

In [ ]:
# example

Q = question_set['questions'][20]
# {'qid': 21,
#  'source': [586, 37, 639, 338, 8, 528, 392, 482, 36],
#  'query': '受益人申領「完全失能保險金」時應檢具哪些文件？',
#  'category': 'insurance'}


filter_conditions = {
    "$and": [
        {"category": {"$eq": f"{Q['category']}"}},
        {"source": {"$in": [str(i) for i in Q['source']]}}
    ]
}
vector_store.similarity_search(Q['query'],filter=filter_conditions,k=4)#[0].metadata


[Document(metadata={'category': 'insurance', 'file_source': 'corpus_insurance.json', 'page_number': 1, 'source': '36', 'source_id': '36'}, page_content='南山人壽新多福保美元利率變動型終身保險（定期給付型）_BUPL5G 約定給付予被保險人： 一、本公司按其經「醫院」診斷確定完全失能當時，分別以「基本保額」及「累 計增額繳清保險基本保額」對應下列三者中之最大值，再依其總和計算「完 全失能保險金」： （一）「當年度保險金額」。 （二）保單價值準備金乘以「當年度保障係數」。 （三）「年繳保險費總和」的一點零一倍。 二、若本契約依第三十二條變更為「展期定期保險」者，本公司按其展期定期保 險金額，計算「完全失能保險金」。 本契約保險費繳費方法為分期繳者，若被保險人於「繳費期間」內經「醫院」診 斷確定為附表二所列完全失能等級之一者，本公司就「基本保額」對應部分另按 日數比例計算當期已繳付之未滿期保險費，退還予要保人。 要保人若依第三十一條辦理「減額繳清保險」或依第三十二條變更為「展期定期 保險」時，不適用前項之約定。 被保險人同時符合附表二所列兩款以上完全失能等級時，本公司僅給付一款「完 全失能保險金」。 本公司依本條約定給付「完全失能保險金」，本契約效力即行終止。但若要保人 有指定分期方式給付「完全失能保險金」，改依第二十五條第一項約定辦理。 第二十二條完全失能保險金的申領 受益人申領「完全失能保險金」時應檢具下列文件： 一、保險單或其謄本。 二、失能診斷書。 三、保險金申請書。 四、受益人的身分證明。 受益人申領「完全失能保險金」時，本公司基於審核保險金之需要，得對被保險 人的身體予以檢驗，另得徵詢其他醫師之醫學專業意見，並得經受益人同意調閱 被保險人之就醫相關資料。因此所生之費用由本公司負擔。 第二十三條特定疾病保險金（嚴重阿茲海默氏症、嚴重巴金森氏症）的給付 被保險人於本契約「保障期間」內診斷確定罹患特定疾病時，本公司按其診斷確 定當時分別以下列二款計算所得之金額，再依其總和給付「特定疾病保險金（嚴 重阿茲海默氏症、嚴重巴金森氏症）」： 一、「基本保額」的百分之五。 二、「累計增額繳清保險基本保額」的百分之五。 前項「

In [ ]:
answers = []

for Q in question_set['questions']:
    filter_conditions = {
        "$and": [
            {"category": {"$eq": f"{Q['category']}"}},
            {"source": {"$in": [str(i) for i in Q['source']]}}
        ]
    }
    chroma_output = vector_store.similarity_search(Q['query'],filter=filter_conditions,k=1)

    if chroma_output:
        query_result = chroma_output[0].metadata
    else:
        print(f"Find no answer for {Q['qid']}")
        query_result = {'category': 'not found',
                        'file_source': 'not found',
                        'page_number': -1,
                        'source': '-1',
                        'source_id': '-1'}

    answers.append({
        'qid': Q['qid'],
        'retrieve': int(query_result['source_id']),
        'category': query_result['category']
    })

with open("./predict_answers_1011.json", "w") as Output:
    json.dump({"answers": answers},Output,ensure_ascii=False,indent=4)
        

Find no answer for 70
Find no answer for 72


In [ ]:
## then use calculateScore.py to calculate Precision@1
## calculateScore.py --ground_truths_path ground_truths_example.json --predictions_path predict_answers_1011.json --output_file ./answer_score.txt